# calculating the RM for each stress-testing scenario

This notebook calucates how different policy is performing in terms of planning reserve margin under different stress testing scenarios 

this notebook take the state of charge for battery resources into account but not the curtailment of solar and wind

# Reconstructing Hourly Energy Availability & Reserve Margin

- **Thermal Resources:** Use `Provide_Power_Capacity_In_Timepoint_MW`, making the assumption that they're not energy-constrained in any way
- **Wind, Solar:** Use `Provide_Power_Capacity_In_Timepoint_MW`, making the claim that curtailment is only an economic decision & not a reliability one (i.e., if energy were needed, renewables could be un-curtailed)
- **Geothermal, Biomass & Hydro:** Use `Provide_Power_MW`, since these have implicit and/or explicit energy limits that may/may not be reflected just by using `Provide_Power_Capacity_In_Timepoint_MW`
- **Storage:** Using just `Provide_Power_MW` does not reflect that storage has energy in storage even in hours that it is not being dispatched. Instead, derive a metric from available state-of-charge in each hour, and there are a few options (in order of under-counting → over-counting):
    1. Just `Provide_Power_MW` is too conservative
  2. **`SOC_Inter_Intra_Joint / duration`: Define energy availability as MW that the storage could discharge at for its entire duration (e.g., an 100 MW, 8-hour storage with 400 MWh SoC in a given hour would have an effective hourly rating of 50 MW).**
        - In Liyang's cases, `SOC_Inter_Period` and `SOC_Inter_Intra_Joint` are the same (since all chrono periods are kept)
        - A minor detail is that the IRP assumes "batteries" have a min SoC of 10%, so the calculation I would do is actually for resources that have that min SoC is `SOC_Inter_Intra_Joint / duration * 0.9`
    3. `min(Provide_Power_Capacity_In_Timepoint_MW, SOC_Inter_Intra_Joint)`: Doesn't matter that the storage can't discharge for multiple hours, if it has enough SoC for a given hour, that's it's energy availability
    4. Just `SOC` or `Provide_Power_Capacity_In_Timepoint_MW` is too optimistic
- **Imports:** If you [read the I&A §7.1.13 Imports](https://www.cpuc.ca.gov/-/media/cpuc-website/divisions/energy-division/documents/integrated-resource-plan-and-long-term-procurement-plan-irp-ltpp/2023-irp-cycle-events-and-materials/inputs-assumptions-2022-2023_final_document_10052023.pdf), the CPUC's `servm` model assumes an hourly shape for imports that can be relied on for RA. So there are actually three different hourly imports you could use (in order of under-counting → over-counting):
    1. **The `resolve` assumed RA imports (which is for some reason that I don't remember 3,100 MW in 2045)**
    2. The `servm` assumed shaped RA imports (11,040 MW, derated to 4,000 MW during summer evenings)
        In SERVM, all units outside CAISO that may deliver energy to CAISO load are subject to SERVM’s
simultaneous import constraint, which is configured as 4,000 MW during peak hours (5pm to
10pm) in June through September, and as 11,040 MW (reflective of the current CAISO
maximum import limit) during all other hours.
    3. The `resolve` actual hourly dispatch (tx limits for economic dispatch are less constrained than the 11,040 MW/5,000 MW assumed

## Reserve Margin
- CAISO uses a 6% contingency reserve assumption (see [Sept 2022 Summer Market Performance Report §3.1](https://www.caiso.com/Documents/SummerMarketPerformanceReportforSeptember2022.pdf)). This differs from the `resolve` assumption
- IRP `resolve` model uses 16% PRM in 2045
- In [my Excel workbook](./outputs/Plots.xlsb), I filtered for days where the reserve margin fell below 6% in any given hour of the day

In [1]:
import os
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from pathlib import Path
import logging

In [2]:
# Load the CSV file into a DataFrame
linkages = pd.read_csv('/Users/liyangwang/kitxDMDU_analysis/2045-Core_test/filtered_linkages.csv')

# Filter the DataFrame
filteredLinkages = linkages[(linkages['linkage'] == 'ResourceToZone') & (linkages['component_to'] == 'CAISO')]

filteredLinkages

,component_from,linkage,scenario,component_to
0,CAISO_Hydro,ResourceToZone,Base,CAISO
1,CAISO_Shed_DR_Existing,ResourceToZone,Shed DR LSE Plans: No LSE Plans,CAISO
2,CAISO_Shed_DR_Existing,ResourceToZone,Shed DR LSE Plans: 25 MMT,CAISO
3,CAISO_Shed_DR_Existing,ResourceToZone,Shed DR LSE Plans: 30 MMT,CAISO
4,Shed_ag_Pumping,ResourceToZone,Base,CAISO
...,...,...,...,...
186,Flex_LDV_V2G_Com,ResourceToZone,Mid_LDV,CAISO
187,Flex_LDV_V1G_Res,ResourceToZone,High_LDV,CAISO
188,Flex_LDV_V1G_Com,ResourceToZone,High_LDV,CAISO
189,Flex_LDV_V2G_Res,ResourceToZone,High_LDV,CAISO


In [3]:
import logging

def setup_logging(log_file_path):
    logging.basicConfig(filename=log_file_path, level=logging.INFO, 
                        format='%(asctime)s - %(levelname)s - %(message)s')

def process_rm(policy_folder, output_file_path, log_file_path):
    setup_logging(log_file_path)
    reserveMarginList = []

    for root, dirs, files in os.walk(policy_folder):
        if 'expressions' in dirs:
            try:
                expressions_dir = os.path.join(root, 'expressions')
                parameters_dir = os.path.join(root, 'parameters')
                rep_periods_dir = os.path.join(root, 'temporal_settings')
                variables_dir = os.path.join(root, 'variables')
                temporal_settings_dir = os.path.join(root, 'temporal_settings')
                
                plant_capacity_file = os.path.join(expressions_dir, 'Plant_Provide_Power_Capacity_In_Timepoint_MW.csv')
                input_load_file = os.path.join(parameters_dir, 'input_load_mw.csv')
                soc_intra_period_file = os.path.join(variables_dir, 'SOC_Intra_Period.csv')
                provide_power_mw_file = os.path.join(variables_dir, 'Provide_Power_MW.csv')
                rep_periods_file = os.path.join(temporal_settings_dir, 'rep_periods.csv')
                transmit_power_mw_file = os.path.join(variables_dir, 'Transmit_Power_MW.csv')

                plantCapacity = pd.read_csv(plant_capacity_file)
                inputLoad = pd.read_csv(input_load_file)
                socIntra = pd.read_csv(soc_intra_period_file)
                providePower = pd.read_csv(provide_power_mw_file)
                repPeriod = pd.read_csv(rep_periods_file)
                transmitPower = pd.read_csv(transmit_power_mw_file)

                logging.info(f"Loaded CSV files from {root}")

                filteredPlantCapacity = plantCapacity[plantCapacity['PLANTS_THAT_PROVIDE_POWER'].isin(filteredLinkages['component_from'])]
                resources_with_storage = socIntra['RESOURCES_WITH_STORAGE'].unique()

                truePlantCapacity = pd.merge(
                    filteredPlantCapacity,
                    providePower,
                    on=['MODEL_YEARS', 'REP_PERIODS', 'HOURS', 'PLANTS_THAT_PROVIDE_POWER'],
                    how='inner'
                )
                mask = truePlantCapacity['PLANTS_THAT_PROVIDE_POWER'].isin(resources_with_storage)
                # finding the storage resources and incorporate state of charge into account, 
                # but this doesn't take curtailment into account for solar and wind resources
                #storage comes from Provide_power_MW
                #everything else comes from Plant_Provide_Power_Capacity_In_Timepoint_MW
                truePlantCapacity.loc[mask, 'Plant_Provide_Power_Capacity_In_Timepoint_MW'] = truePlantCapacity.loc[mask, 'Provide_Power_MW']

                try:
                    truePlantCapacity = pd.merge(truePlantCapacity, repPeriod[['period', '0']], left_on='REP_PERIODS', right_on='period', how='left')
                    truePlantCapacity['datetime'] = pd.to_datetime(truePlantCapacity['MODEL_YEARS'].astype(str) + '-' + truePlantCapacity['0'].str[5:], errors='coerce')
                    truePlantCapacity['datetime'] = truePlantCapacity['datetime'] + pd.to_timedelta(truePlantCapacity['HOURS'], unit='h')
                    truePlantCapacity.drop(columns=['period', '0'], inplace=True)
                    logging.info("Converted MODEL_YEARS, REP_PERIODS, and HOURS into datetime successfully.")
                except Exception as e:
                    logging.error(f"Error converting MODEL_YEARS, REP_PERIODS, and HOURS into datetime: {e}")

                try:
                    caisoLoad = inputLoad[inputLoad['ZONES'] == 'CAISO'].copy()
                    caisoLoad[['REP_PERIODS', 'HOURS', 'MODEL_YEARS']] = caisoLoad[['REP_PERIODS', 'HOURS', 'MODEL_YEARS']].astype(int)
                    caisoLoad = pd.merge(caisoLoad, repPeriod[['period', '0']], left_on='REP_PERIODS', right_on='period', how='left')
                    caisoLoad['datetime'] = pd.to_datetime(caisoLoad['MODEL_YEARS'].astype(str) + '-' + caisoLoad['0'].str[5:], errors='coerce') + pd.to_timedelta(caisoLoad['HOURS'], unit='h')
                    caisoLoad = caisoLoad.sort_values(by='datetime')
                    logging.info("Calculated hourly input load successfully.")
                except Exception as e:
                    logging.error(f"Error calculating hourly input load: {e}")
                    
                try:
                    caisoImportExport = transmitPower[transmitPower['TRANSMISSION_LINES'].str.contains('to CAISO')].copy()
                    caisoImportExport[['REP_PERIODS', 'HOURS', 'MODEL_YEARS']] = caisoImportExport[['REP_PERIODS', 'HOURS', 'MODEL_YEARS']].astype(int)
                    caisoImportExport = pd.merge(caisoImportExport, repPeriod[['period', '0']], left_on='REP_PERIODS', right_on='period', how='left')
                    caisoImportExport['datetime'] = pd.to_datetime(caisoImportExport['MODEL_YEARS'].astype(str) + '-' + caisoImportExport['0'].str[5:], errors='coerce') + pd.to_timedelta(caisoImportExport['HOURS'], unit='h')
                    caisoImportExport = caisoImportExport.sort_values(by='datetime')
                    caisoImportExport = caisoImportExport.groupby('datetime')['Transmit_Power_MW'].sum().reset_index()
                    logging.info("Calculated hourly import/export successfully.")
                except Exception as e:
                    logging.error(f"Error calculating import/export : {e}")

                try:
                    reserveMargin = truePlantCapacity.groupby('datetime')['Plant_Provide_Power_Capacity_In_Timepoint_MW'].sum().reset_index()
                    reserveMargin = reserveMargin.merge(caisoImportExport, on='datetime')
                    reserveMargin = reserveMargin.merge(caisoLoad, on='datetime')
                    reserveMargin['PRM'] = ((reserveMargin['Plant_Provide_Power_Capacity_In_Timepoint_MW'] + reserveMargin['Transmit_Power_MW']  - reserveMargin['input_load_mw']) / reserveMargin['input_load_mw']) * 100
                    logging.info("Calculated reserve margin successfully.")
                except Exception as e:
                    logging.error(f"Error calculating reserve margin: {e}")

                parts = root.split('/')
                part1 = parts[-3].replace('2010-2020ML', '-')
                part2 = parts[-2].replace('2045-', '')
                case_name = f"{part1}{part2}"
                reserveMargin['case'] = case_name
                reserveMarginList.append(reserveMargin)
            except FileNotFoundError as e:
                logging.error(f"File not found: {e}")
                continue

    RMall_df = pd.concat(reserveMarginList, ignore_index=True)
    RMall_df.drop(columns=['ZONES', 'MODEL_YEARS', 'REP_PERIODS', 'HOURS', 'period', '0'], inplace=True)
    RMall_df.to_csv(output_file_path, index=False)
    logging.info(f"Saved RMall_df to {log_file_path}")
    return RMall_df

In [ ]:
def setup_logging(log_file_path):
    logging.basicConfig(filename=log_file_path, level=logging.INFO, 
                        format='%(asctime)s - %(levelname)s - %(message)s')

def process_rm(policy_folder, output_file_path, log_file_path):
    setup_logging(log_file_path)
    reserveMarginList = []

    resource_multiplier = pd.read_csv('/Users/liyangwang/kitxDMDU_analysis/resourceListMultiplier.csv')

    for root, dirs, files in os.walk(policy_folder):
        if 'expressions' in dirs:
            try:
                expressions_dir = os.path.join(root, 'expressions')
                parameters_dir = os.path.join(root, 'parameters')
                rep_periods_dir = os.path.join(root, 'temporal_settings')
                variables_dir = os.path.join(root, 'variables')
                temporal_settings_dir = os.path.join(root, 'temporal_settings')
                
                plant_capacity_file = os.path.join(expressions_dir, 'Plant_Provide_Power_Capacity_In_Timepoint_MW.csv')
                input_load_file = os.path.join(parameters_dir, 'input_load_mw.csv')
                soc_intra_period_file = os.path.join(variables_dir, 'SOC_Intra_Period.csv')
                provide_power_mw_file = os.path.join(variables_dir, 'Provide_Power_MW.csv')
                rep_periods_file = os.path.join(temporal_settings_dir, 'rep_periods.csv')
                transmit_power_mw_file = os.path.join(variables_dir, 'Transmit_Power_MW.csv')

                plantCapacity = pd.read_csv(plant_capacity_file)
                inputLoad = pd.read_csv(input_load_file)
                socIntra = pd.read_csv(soc_intra_period_file)
                providePower = pd.read_csv(provide_power_mw_file)
                repPeriod = pd.read_csv(rep_periods_file)
                transmitPower = pd.read_csv(transmit_power_mw_file)

                logging.info(f"Loaded CSV files from {root}")

                filteredPlantCapacity = plantCapacity[plantCapacity['PLANTS_THAT_PROVIDE_POWER'].isin(resource_multiplier['Resources'])]
                resources_with_storage = socIntra['RESOURCES_WITH_STORAGE'].unique()

                truePlantCapacity = pd.merge(
                    filteredPlantCapacity,
                    providePower,
                    on=['MODEL_YEARS', 'REP_PERIODS', 'HOURS', 'PLANTS_THAT_PROVIDE_POWER'],
                    how='inner'
                )

                for index, row in resource_multiplier.iterrows():
                    resource = row['Resources']
                    category = row['Category']
                    multiplier = row['Multiplier']
                    if category in ['Gas', 'Nuclear', 'Solar', 'Wind']:
                        truePlantCapacity.loc[truePlantCapacity['PLANTS_THAT_PROVIDE_POWER'] == resource, 'Plant_Provide_Power_Capacity_In_Timepoint_MW'] = truePlantCapacity['Provide_Power_Capacity_In_Timepoint_MW']
                    elif category in ['Bio/Geo', 'Hydro']:
                        truePlantCapacity.loc[truePlantCapacity['PLANTS_THAT_PROVIDE_POWER'] == resource, 'Plant_Provide_Power_Capacity_In_Timepoint_MW'] = truePlantCapacity['Provide_Power_MW']
                    elif category == 'Storage':
                        soc_value = socIntra[socIntra['RESOURCES_WITH_STORAGE'] == resource]['SOC_Intra_Period'].values[0]
                        truePlantCapacity.loc[truePlantCapacity['PLANTS_THAT_PROVIDE_POWER'] == resource, 'Plant_Provide_Power_Capacity_In_Timepoint_MW'] = soc_value * multiplier

                try:
                    truePlantCapacity = pd.merge(truePlantCapacity, repPeriod[['period', '0']], left_on='REP_PERIODS', right_on='period', how='left')
                    truePlantCapacity['datetime'] = pd.to_datetime(truePlantCapacity['MODEL_YEARS'].astype(str) + '-' + truePlantCapacity['0'].str[5:], errors='coerce')
                    truePlantCapacity['datetime'] = truePlantCapacity['datetime'] + pd.to_timedelta(truePlantCapacity['HOURS'], unit='h')
                    truePlantCapacity.drop(columns=['period', '0'], inplace=True)
                    logging.info("Converted MODEL_YEARS, REP_PERIODS, and HOURS into datetime successfully.")
                except Exception as e:
                    logging.error(f"Error converting MODEL_YEARS, REP_PERIODS, and HOURS into datetime: {e}")

                try:
                    caisoLoad = inputLoad[inputLoad['ZONES'] == 'CAISO'].copy()
                    caisoLoad[['REP_PERIODS', 'HOURS', 'MODEL_YEARS']] = caisoLoad[['REP_PERIODS', 'HOURS', 'MODEL_YEARS']].astype(int)
                    caisoLoad = pd.merge(caisoLoad, repPeriod[['period', '0']], left_on='REP_PERIODS', right_on='period', how='left')
                    caisoLoad['datetime'] = pd.to_datetime(caisoLoad['MODEL_YEARS'].astype(str) + '-' + caisoLoad['0'].str[5:], errors='coerce') + pd.to_timedelta(caisoLoad['HOURS'], unit='h')
                    caisoLoad = caisoLoad.sort_values(by='datetime')
                    logging.info("Calculated hourly input load successfully.")
                except Exception as e:
                    logging.error(f"Error calculating hourly input load: {e}")
                    
                try:
                    caisoImportExport = transmitPower[transmitPower['TRANSMISSION_LINES'].str.contains('to CAISO')].copy()
                    caisoImportExport[['REP_PERIODS', 'HOURS', 'MODEL_YEARS']] = caisoImportExport[['REP_PERIODS', 'HOURS', 'MODEL_YEARS']].astype(int)
                    caisoImportExport = pd.merge(caisoImportExport, repPeriod[['period', '0']], left_on='REP_PERIODS', right_on='period', how='left')
                    caisoImportExport['datetime'] = pd.to_datetime(caisoImportExport['MODEL_YEARS'].astype(str) + '-' + caisoImportExport['0'].str[5:], errors='coerce') + pd.to_timedelta(caisoImportExport['HOURS'], unit='h')
                    caisoImportExport = caisoImportExport.sort_values(by='datetime')
                    caisoImportExport = caisoImportExport.groupby('datetime')['Transmit_Power_MW'].sum().reset_index()
                    logging.info("Calculated hourly import/export successfully.")
                except Exception as e:
                    logging.error(f"Error calculating import/export : {e}")

                try:
                    reserveMargin = truePlantCapacity.groupby('datetime')['Plant_Provide_Power_Capacity_In_Timepoint_MW'].sum().reset_index()
                    reserveMargin = reserveMargin.merge(caisoImportExport, on='datetime')
                    reserveMargin = reserveMargin.merge(caisoLoad, on='datetime')
                    reserveMargin['PRM'] = ((reserveMargin['Plant_Provide_Power_Capacity_In_Timepoint_MW'] + reserveMargin['Transmit_Power_MW']  - reserveMargin['input_load_mw']) / reserveMargin['input_load_mw']) * 100
                    logging.info("Calculated reserve margin successfully.")
                except Exception as e:
                    logging.error(f"Error calculating reserve margin: {e}")

                parts = root.split('/')
                part1 = parts[-3].replace('2010-2020ML', '-')
                part2 = parts[-2].replace('2045-', '')
                case_name = f"{part1}{part2}"
                reserveMargin['case'] = case_name
                reserveMarginList.append(reserveMargin)
            except FileNotFoundError as e:
                logging.error(f"File not found: {e}")
                continue

    RMall_df = pd.concat(reserveMarginList, ignore_index=True)
    RMall_df.drop(columns=['ZONES', 'MODEL_YEARS', 'REP_PERIODS', 'HOURS', 'period', '0'], inplace=True)
    RMall_df.to_csv(output_file_path, index=False)
    logging.info(f"Saved RMall_df to {log_file_path}")
    return RMall_df

In [4]:
Core = '/Users/liyangwang/kitxDMDU_analysis/2045-Core_test'  # Replace with your root directory
CoreLog = '/Users/liyangwang/kitxDMDU_analysis/coretest.log'
CorePRM = process_rm(Core, '/Users/liyangwang/kitxDMDU_analysis/2045-Core_test_PRM.csv',CoreLog)


CorePRM

,datetime,Plant_Provide_Power_Capacity_In_Timepoint_MW,Transmit_Power_MW,input_load_mw,PRM,case
0,2045-01-01 00:00:00,47953.201,905.843,46159.888,5.847406,Baseline2_45_CESM2_2016
1,2045-01-01 01:00:00,47488.039,635.384,44710.867,7.632498,Baseline2_45_CESM2_2016
2,2045-01-01 02:00:00,41715.592,1328.683,39829.731,8.070715,Baseline2_45_CESM2_2016
3,2045-01-01 03:00:00,37266.118,2301.159,35554.393,11.286605,Baseline2_45_CESM2_2016
4,2045-01-01 04:00:00,35455.182,1756.286,33076.003,12.502916,Baseline2_45_CESM2_2016
...,...,...,...,...,...,...
26275,2045-12-31 19:00:00,37705.291,3592.131,37700.782,9.539961,Baseline2_45_CESM2_2017
26276,2045-12-31 20:00:00,38039.663,3254.343,37753.060,9.379229,Baseline2_45_CESM2_2017
26277,2045-12-31 21:00:00,38705.301,3622.192,39128.303,8.176153,Baseline2_45_CESM2_2017
26278,2045-12-31 22:00:00,36950.061,4537.209,39109.491,6.079800,Baseline2_45_CESM2_2017


In [5]:
Core = '/global/scratch/users/liyangwang/Robustness/reports/resolve/2045-Core_25MMT'  # Replace with your root directory
highGas = '/global/scratch/users/liyangwang/Robustness/reports/resolve/2045-High_Gas_Ret'  # Replace with your root directory
leastCost = '/global/scratch/users/liyangwang/Robustness/reports/resolve/2045-LeastCost_25MMT_Sens'  # Replace with your root directory

In [6]:
CoreLog = '/global/scratch/users/liyangwang/Robustness/z.output/PRMv2/core.log'
HGLog = '/global/scratch/users/liyangwang/Robustness/z.output/PRMv2/highGas.log'
LCLog = '/global/scratch/users/liyangwang/Robustness/z.output/PRMv2/leastCost.log'

In [7]:
CorePRM = process_rm(Core, '/global/scratch/users/liyangwang/Robustness/z.output/PRMv2/2045-Core_25MMT_PRM.csv',CoreLog)
highGasPRM = process_rm(highGas, '/global/scratch/users/liyangwang/Robustness/z.output/PRMv2/2045-High_Gas_Ret_PRM.csv', HGLog)
leastCostPRM = process_rm(leastCost, '/global/scratch/users/liyangwang/Robustness/z.output/PRMv2/2045-LeastCost_25MMT_Sens_PRM.csv',LCLog)


In [50]:
vulnerabeHours = CorePRM[CorePRM['PRM'] < 0].groupby('case').size().reset_index(name='hours_below_17_prm')

vulnerabeHours

                            case  hours_below_17_prm
0        Baseline2_45_CESM2_2015                1102
1        Baseline2_45_CESM2_2016                 821
2        Baseline2_45_CESM2_2017                 903
3  highDemandslim2_45_CESM2_2015                5620


In [8]:
transmitPower

,TRANSMISSION_LINES,MODEL_YEARS,REP_PERIODS,HOURS,Transmit_Power_MW
0,BANC to CAISO,2045,0,0,1231.761
1,BANC to CAISO,2045,0,1,1918.330
2,BANC to CAISO,2045,0,2,2041.173
3,BANC to CAISO,2045,0,3,1437.308
4,BANC to CAISO,2045,0,4,1370.046
...,...,...,...,...,...
122635,SW to LDWP,2045,364,19,0.000
122636,SW to LDWP,2045,364,20,0.000
122637,SW to LDWP,2045,364,21,0.000
122638,SW to LDWP,2045,364,22,0.000


# debug zone

In [4]:
root ='/Users/liyangwang/kitxDMDU_analysis/2045-Core_test/Baseline/2045-2_45_CESM2_2015/2024-08-16 10-48-13'

resource_multiplier = pd.read_csv('/Users/liyangwang/kitxDMDU_analysis/resourceListMultiplierv2.csv')


In [25]:
expressions_dir = os.path.join(root, 'expressions')
parameters_dir = os.path.join(root, 'parameters')
rep_periods_dir = os.path.join(root, 'temporal_settings')
variables_dir = os.path.join(root, 'variables')
temporal_settings_dir = os.path.join(root, 'temporal_settings')

plant_capacity_file = os.path.join(expressions_dir, 'Plant_Provide_Power_Capacity_In_Timepoint_MW.csv')
input_load_file = os.path.join(parameters_dir, 'input_load_mw.csv')
soc_intra_period_file = os.path.join(variables_dir, 'SOC_Intra_Period.csv')
provide_power_mw_file = os.path.join(variables_dir, 'Provide_Power_MW.csv')
rep_periods_file = os.path.join(temporal_settings_dir, 'rep_periods.csv')
transmit_power_mw_file = os.path.join(variables_dir, 'Transmit_Power_MW.csv')

plantCapacity = pd.read_csv(plant_capacity_file)
inputLoad = pd.read_csv(input_load_file)
socIntra = pd.read_csv(soc_intra_period_file)
providePower = pd.read_csv(provide_power_mw_file)
repPeriod = pd.read_csv(rep_periods_file)
transmitPower = pd.read_csv(transmit_power_mw_file)

logging.info(f"Loaded CSV files from {root}")

# Step 1: Filter plantCapacity
filteredPlantCapacity = plantCapacity[plantCapacity['PLANTS_THAT_PROVIDE_POWER'].isin(resource_multiplier['Resources'])]

# Step 2: Get unique resources with storage
resources_with_storage = socIntra['RESOURCES_WITH_STORAGE'].unique()

# Step 3: Merge filteredPlantCapacity with providePower
truePlantCapacity = pd.merge(
    filteredPlantCapacity,
    providePower,
    on=['MODEL_YEARS', 'REP_PERIODS', 'HOURS', 'PLANTS_THAT_PROVIDE_POWER'],
    how='inner'
)

# Step 4: Merge truePlantCapacity with socIntra
truePlantCapacity = pd.merge(
    truePlantCapacity,
    socIntra[['MODEL_YEARS', 'REP_PERIODS', 'HOURS', 'RESOURCES_WITH_STORAGE', 'SOC_Intra_Period']],
    left_on=['MODEL_YEARS', 'REP_PERIODS', 'HOURS', 'PLANTS_THAT_PROVIDE_POWER'],
    right_on=['MODEL_YEARS', 'REP_PERIODS', 'HOURS', 'RESOURCES_WITH_STORAGE'],
    how='left'
)

for index, row in resource_multiplier.iterrows():
    resource = row['Resources']
    category = row['Category']
    multiplier = row['Multiplier']
    if category in ['Gas', 'Nuclear', 'Solar', 'Wind']:
        truePlantCapacity.loc[truePlantCapacity['PLANTS_THAT_PROVIDE_POWER'] == resource, 'true_capacity'] = truePlantCapacity['Plant_Provide_Power_Capacity_In_Timepoint_MW']
    elif category in ['Bio/Geo', 'Hydro']:
        truePlantCapacity.loc[truePlantCapacity['PLANTS_THAT_PROVIDE_POWER'] == resource, 'true_capacity'] = truePlantCapacity['Provide_Power_MW']
    elif category == 'Storage':
        #soc_value = socIntra[socIntra['RESOURCES_WITH_STORAGE'] == resource]['SOC_Intra_Period'].values[0]
        truePlantCapacity.loc[truePlantCapacity['PLANTS_THAT_PROVIDE_POWER'] == resource, 'true_capacity'] = truePlantCapacity['SOC_Intra_Period'] * multiplier

try:
    truePlantCapacity = pd.merge(truePlantCapacity, repPeriod[['period', '0']], left_on='REP_PERIODS', right_on='period', how='left')
    truePlantCapacity['datetime'] = pd.to_datetime(truePlantCapacity['MODEL_YEARS'].astype(str) + '-' + truePlantCapacity['0'].str[5:], errors='coerce')
    truePlantCapacity['datetime'] = truePlantCapacity['datetime'] + pd.to_timedelta(truePlantCapacity['HOURS'], unit='h')
    truePlantCapacity.drop(columns=['period', '0'], inplace=True)
    logging.info("Converted MODEL_YEARS, REP_PERIODS, and HOURS into datetime successfully.")
except Exception as e:
    logging.error(f"Error converting MODEL_YEARS, REP_PERIODS, and HOURS into datetime: {e}")

In [24]:
# Step 1: Extract unique datetime values
unique_datetimes = truePlantCapacity['datetime'].unique()

# Step 2: Initialize the true_capacity values
capacity_values = []

for dt in unique_datetimes:
    dt = pd.to_datetime(dt)
    if dt.month in [6, 7, 8, 9] and dt.hour >= 17 and dt.hour < 22:
        capacity_values.append(4000)
    else:
        capacity_values.append(11040)

# Step 3: Create a DataFrame with the resulting data
imports = pd.DataFrame({
    'datetime': unique_datetimes,
    'import_capacity': capacity_values
})

# Display the first few rows of the DataFrame
imports.to_csv('/Users/liyangwang/kitxDMDU_analysis/2045-Core_test/imports.csv', index=False)

In [11]:
truePlantCapacity.to_csv('/Users/liyangwang/kitxDMDU_analysis/truePlantCapacity.csv', index=False)

In [26]:
try:
    caisoLoad = inputLoad[inputLoad['ZONES'] == 'CAISO'].copy()
    caisoLoad[['REP_PERIODS', 'HOURS', 'MODEL_YEARS']] = caisoLoad[['REP_PERIODS', 'HOURS', 'MODEL_YEARS']].astype(int)
    caisoLoad = pd.merge(caisoLoad, repPeriod[['period', '0']], left_on='REP_PERIODS', right_on='period', how='left')
    caisoLoad['datetime'] = pd.to_datetime(caisoLoad['MODEL_YEARS'].astype(str) + '-' + caisoLoad['0'].str[5:], errors='coerce') + pd.to_timedelta(caisoLoad['HOURS'], unit='h')
    caisoLoad = caisoLoad.sort_values(by='datetime')
    logging.info("Calculated hourly input load successfully.")
except Exception as e:
    logging.error(f"Error calculating hourly input load: {e}")
    

try:
    reserveMargin = truePlantCapacity.groupby('datetime')['true_capacity'].sum().reset_index()
    reserveMargin = reserveMargin.merge(imports, on='datetime')
    reserveMargin = reserveMargin.merge(caisoLoad, on='datetime')
    reserveMargin['PRM'] = ((reserveMargin['true_capacity'] + reserveMargin['import_capacity']  - reserveMargin['input_load_mw']) / reserveMargin['input_load_mw']) * 100
    logging.info("Calculated reserve margin successfully.")
except Exception as e:
    logging.error(f"Error calculating reserve margin: {e}")

parts = root.split('/')
part1 = parts[-3].replace('2010-2020ML', '-')
part2 = parts[-2].replace('2045-', '')
case_name = f"{part1}{part2}"
reserveMargin['case'] = case_name
#reserveMarginList.append(reserveMargin)

In [28]:
reserveMargin.to_csv('/Users/liyangwang/kitxDMDU_analysis/PRM.csv', index=False)